# 1. Get the packages

In [37]:
import cohere
import sys,os
import pandas as pd
import requests
import datetime
from tqdm import tqdm

In [42]:
sys.path.append(os.path.abspath(os.path.join("../scripts")))
from preprocess import Preporcess
from prompt_generation import cohereExtractor
from prompt_preprocess import TestPreprocess

prompt_prp = TestPreprocess()
prp = Preporcess()


# 2. Get the Data

In [43]:
from cgi import test

train_df = pd.read_json('../data/entity/relations_training.txt')
test_df = pd.read_json('../data/entity/relations_test.txt') 


In [44]:
train_df['label'] = train_df['tokens'].apply(prp.preprocess_tokens)
test_df['label'] = test_df['tokens'].apply(prp.preprocess_tokens)


In [45]:
test_df

,document,tokens,relations,label
0,"\nCurrently holding a faculty, industry, or go...","[{'text': 'Ph.D.', 'start': 75, 'end': 80, 'to...","[{'child': 18, 'head': 14, 'relationLabel': 'D...","DIPLOMA:Ph.D.\nDIPLOMA_MAJOR:machine learning,..."
1,\n2+ years experience in the online advertisin...,"[{'text': '2+ years', 'start': 1, 'end': 9, 't...","[{'child': 7, 'head': 1, 'relationLabel': 'EXP...",EXPERIENCE:2+ years\nSKILLS:online advertising...
2,\nBA/BS\n5+ years of program or project manage...,"[{'text': '5+ years', 'start': 7, 'end': 15, '...","[{'child': 11, 'head': 5, 'relationLabel': 'EX...","EXPERIENCE:5+ years,2+ years\nSKILLS:project m..."
3,\nCurrently enrolled in a full-time degree pro...,"[{'text': 'Ph.D.', 'start': 801, 'end': 806, '...","[{'child': 140, 'head': 137, 'relationLabel': ...",DIPLOMA:Ph.D.\nDIPLOMA_MAJOR:Computer Science\n
4,\nCurrently enrolled in a full-time degree pro...,"[{'text': 'Ph.D.', 'start': 801, 'end': 806, '...","[{'child': 140, 'head': 137, 'relationLabel': ...",DIPLOMA:Ph.D.\nDIPLOMA_MAJOR:Computer Science\n
5,\nMS in a quantitative field such as Operation...,"[{'text': 'MS', 'start': 1, 'end': 3, 'token_s...","[{'child': 8, 'head': 1, 'relationLabel': 'DEG...","DIPLOMA:MS,PhD\nDIPLOMA_MAJOR:Operations Resea..."
6,"\nPh.D. with 5+ years of experience, MS with 7...",[],[],
7,\nGraduating with a Ph.D. in Computer Science ...,"[{'text': 'Ph.D.', 'start': 19, 'end': 24, 'to...","[{'child': 6, 'head': 4, 'relationLabel': 'DEG...",DIPLOMA:Ph.D.\nDIPLOMA_MAJOR:Computer Science\n
8,"\n12+ years of research experience, in-house o...","[{'text': '12+ years', 'start': 1, 'end': 10, ...","[{'child': 5, 'head': 1, 'relationLabel': 'EXP...","EXPERIENCE:12+ years\nSKILLS:research,Ph.D\nDI..."
9,"\nPhD with 3+ years, or MS with 5+ years of ex...","[{'text': '5+ years', 'start': 31, 'end': 39, ...","[{'child': 18, 'head': 10, 'relationLabel': 'E...","EXPERIENCE:5+ years,3+ years,3+ years,5+ years..."


# 3. preprocess the data

In [20]:

train_doc = []
for i in range(train_df.shape[0]):
    ent = train_df.label.iloc[i]
    docu = train_df.document.iloc[i].replace("\n"," ")
    if len(ent)!=0:
        train_doc.append(docu+"\n\nExtracted Text:" +
                        '\n'+ent+"---\n")
    


Done


In [46]:
train_doc = prp.preprocess_document(train_df)

with open('../data/output/entity.txt', 'w') as f:
    for item in train_doc:
        # write each item on a new line
        f.write("%s\n" % item.strip())
    print('Done')


Done


In [47]:

test_doc = prompt_prp.entity_extraction(test_df)

# test_df.document.apply(
    # lambda x: x.replace("\n", " ")+'\n\nExtracted Text:').to_list()


In [48]:
import cohere
api_key = pd.read_json("../config/apikey.json")['cohere_key'][0]
co = cohere.Client(api_key)


# 4. Running the model

In [50]:
cohereMovieExtractor = cohereExtractor(train_doc[:6],co)


In [51]:
print(cohereMovieExtractor.make_prompt(test_doc[-1]))


MS/Ph.D. in CS/EE or related areas is required. Strong ability and effectiveness working in a significant technical problem domain, in the term of plan, design, execution, continuous release and service operation. Strong software engineering fundamentals, including coding, problem solving and data analysis skills. Background in machine learning/deep learning (strongly preferred). Passionate and self-motivated. Ability to effectively work in collaborative multiple project team environment and ship production features in a fast-paced environment. Good communication skills, both verbal and written. Customer/end result/Metrics driven in design and development. Strong ability in self-learning, entering new domain, managing through uncertainty in an innovative team environment

Extracted Text:
DIPLOMA:MS/Ph.D.
DIPLOMA_MAJOR:CS/EE
----
1+ years development experience on Java stack AppConnect / API's experience is added advantage. Compute, Network and Storage Monitoring Tools (Ex: Netcool) App

In [52]:
results = []
for text in tqdm(test_doc):
    try:
        extracted_text = cohereMovieExtractor.extract(text)
        print(extracted_text)
        results.append(extracted_text)
    except Exception as e:
        print('ERROR: ', e)


  9%|▉         | 1/11 [00:04<00:49,  4.93s/it]


DIPLOMA:PhD
DIPLOMA_MAJOR:Machine Learning,AI,Computer Science,Statistics,Applied Mathematics,Data Science,Quantitative
---


 18%|█▊        | 2/11 [00:08<00:37,  4.22s/it]


DIPLOMA:Masters,MBA
DIPLOMA_MAJOR:Statistics,Political Science,History,Psychology
---


 27%|██▋       | 3/11 [00:12<00:30,  3.87s/it]

...


EXPERIENCE:12+ years
SKILLS:research experience,in-house,vendor,applicable consulting experience
DIPLOMA:PhD
DIPLOMA_MAJOR:Human-computer interaction,Human factors,cognitive psychology
---


 91%|█████████ | 10/11 [00:45<00:04,  4.31s/it]


EXPERIENCE:3+ years
SKILLS:people research,organization
DIPLOMA:PhD
DIPLOMA_MAJOR:Industrial/Organizational Psychology,Organizational Behavior,Labor Economics,Management,Policy Analysis,
---


100%|██████████| 11/11 [00:51<00:00,  4.67s/it]


EXPERIENCE:1+ years
SKILLS:research,researcher
---


In [53]:
pd.DataFrame(data={'text': test_doc, 'extracted_text': results})


,text,extracted_text
0,"Currently holding a faculty, industry, or gov...","\nDIPLOMA:PhD\nDIPLOMA_MAJOR:Machine Learning,..."
1,2+ years experience in the online advertising...,"\nDIPLOMA:Masters,MBA\nDIPLOMA_MAJOR:Statistic..."
2,BA/BS 5+ years of program or project manageme...,\nDIPLOMA:BA/BS\nDIPLOMA_MAJOR:Program Managem...
3,Currently enrolled in a full-time degree prog...,"\nEXPERIENCE:2-3 years\nSKILLS:research,data s..."
4,Currently enrolled in a full-time degree prog...,\nDIPLOMA:PhD\nDIPLOMA_MAJOR:Computer Science\...
5,MS in a quantitative field such as Operations...,"\nEXPERIENCE:7+ years\nSKILLS:statistics,algor..."
6,"Ph.D. with 5+ years of experience, MS with 7+...","\nEXPERIENCE:5+ years\nSKILLS:Physics,Electric..."
7,Graduating with a Ph.D. in Computer Science o...,\nDIPLOMA:PhD\nDIPLOMA_MAJOR:Computer Science\...
8,"12+ years of research experience, in-house or...",\nEXPERIENCE:12+ years\nSKILLS:research experi...
9,"PhD with 3+ years, or MS with 5+ years of exp...","\nEXPERIENCE:3+ years\nSKILLS:people research,..."


In [54]:
from concurrent.futures import ThreadPoolExecutor

extracted = []
# Run the model to extract the entities
with ThreadPoolExecutor(max_workers=8) as executor:
    for i in executor.map(cohereMovieExtractor.extract, test_df['document'].apply(
            lambda x: "Job: "+x.replace("\n", " ")+'\n\nExtracted Text:').to_list()):
        extracted.append(str(i).strip())



# Evaluation

In [55]:
# Save results
test_df['extracted_text'] = extracted


In [57]:
test_df['extracted_text'] = test_df['extracted_text'].apply(prp.clean_extracted_token)
test_df['label'] = test_df['label'].apply(prp.clean_test_token)


In [58]:
from difflib import SequenceMatcher
score = []
for i in range(test_df.shape[0]):
    
    score.append(SequenceMatcher(None,test_df['label'].iloc[i],
                          test_df['extracted_text'].iloc[i]).ratio())

In [59]:
# Compare the label to the extracted text
test_df['similarity_score'] = pd.DataFrame(score)

# Print the accuracy
print(f'Classification accuracy {test_df["similarity_score"].mean() *100}%')


Classification accuracy 49.329253843633055%


In [60]:
test_df

,document,tokens,relations,label,extracted_text,similarity_score
0,"\nCurrently holding a faculty, industry, or go...","[{'text': 'Ph.D.', 'start': 75, 'end': 80, 'to...","[{'child': 18, 'head': 14, 'relationLabel': 'D...","diploma:ph.d.;diploma_major:machine learning,a...","diploma:phd;diploma_major:machine learning,ai,...",0.722581
1,\n2+ years experience in the online advertisin...,"[{'text': '2+ years', 'start': 1, 'end': 9, 't...","[{'child': 7, 'head': 1, 'relationLabel': 'EXP...","experience:2+ years;skills:online advertising,...","experience:2+ years;skills:online advertising,...",0.508621
2,\nBA/BS\n5+ years of program or project manage...,"[{'text': '5+ years', 'start': 7, 'end': 15, '...","[{'child': 11, 'head': 5, 'relationLabel': 'EX...","experience:5+ years,2+ years;skills:project ma...",diploma:ba/bs;diploma_major:2+ years;,0.186047
3,\nCurrently enrolled in a full-time degree pro...,"[{'text': 'Ph.D.', 'start': 801, 'end': 806, '...","[{'child': 140, 'head': 137, 'relationLabel': ...",diploma:ph.d.;diploma_major:computer science;,diploma:phd;diploma_major:computer science;,0.977273
4,\nCurrently enrolled in a full-time degree pro...,"[{'text': 'Ph.D.', 'start': 801, 'end': 806, '...","[{'child': 140, 'head': 137, 'relationLabel': ...",diploma:ph.d.;diploma_major:computer science;,diploma:phd;diploma_major:computer science;,0.977273
5,\nMS in a quantitative field such as Operation...,"[{'text': 'MS', 'start': 1, 'end': 3, 'token_s...","[{'child': 8, 'head': 1, 'relationLabel': 'DEG...","diploma:ms,phd;diploma_major:operations resear...",diploma:ms;diploma_major:industrial engineering;,0.192771
6,"\nPh.D. with 5+ years of experience, MS with 7...",[],[],,"diploma:ph.d.,ms,bs;diploma_major:physics,elec...",0.000000
7,\nGraduating with a Ph.D. in Computer Science ...,"[{'text': 'Ph.D.', 'start': 19, 'end': 24, 'to...","[{'child': 6, 'head': 4, 'relationLabel': 'DEG...",diploma:ph.d.;diploma_major:computer science;,"diploma:phd;diploma_major:computer science,rel...",0.843137
8,"\n12+ years of research experience, in-house o...","[{'text': '12+ years', 'start': 1, 'end': 10, ...","[{'child': 5, 'head': 1, 'relationLabel': 'EXP...","experience:12+ years;skills:research,ph.d;dipl...",experience:12+ years;skills:research experienc...,0.449198
9,"\nPhD with 3+ years, or MS with 5+ years of ex...","[{'text': '5+ years', 'start': 31, 'end': 39, ...","[{'child': 18, 'head': 10, 'relationLabel': 'E...","experience:5+ years,3+ years,3+ years,5+ years...","experience:3+ years;skills:people research,emp...",0.283604
